#HW3 - SQL
 
This homework has you working with a new database of information on ticket sales for various types of events.  Your job will be to do some initial exploring and then demonstrate your ability to do all the different types of SQL queries we learned over the last two week.  You'll also need to make one function that'll make looking at the tables easier. 
 
These questions are written in the way someone would ask them to you.  In other words, I'm using 'plain english' questions vs. ones where I'm very explicit in terms of what columns and tables to use.  Your exploring of the database and functions to ease that process will come in handy here!  
 
Also, I made the database using a set of data from Amazon. You can read more about what each table contains here: https://docs.aws.amazon.com/redshift/latest/dg/c_sampledb.html.  

## Libraries and import functions

First bring the libraries we'll need!

In [1]:
import psycopg2 
import pandas as pd

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Now bring in all our functions we used in the lessons!  

In [2]:
# Make our connection/cursor function 

def get_conn_cur(): # define function name and arguments (there aren't any)
  # Make a connection
  conn = psycopg2.connect(
    host="ticketsdbinstance.cnt7cm8tgir1.us-west-1.rds.amazonaws.com",
    database="sales_db",
    user="postgres",
    password="ISTA322SALESDB",
    port='5432')
  
  cur = conn.cursor()   # Make a cursor after

  return(conn, cur)   # Return both the connection and the cursor

# Same run_query function
def run_query(query_string):

  conn, cur = get_conn_cur() # get connection and cursor

  cur.execute(query_string) # executing string as before

  my_data = cur.fetchall() # fetch query data as before

  # here we're extracting the 0th element for each item in cur.description
  colnames = [desc[0] for desc in cur.description]

  cur.close() # close
  conn.close() # close

  return(colnames, my_data) # return column names AND data

# Column name function for checking out what's in a table
def get_column_names(table_name): # arguement of table_name
  conn, cur = get_conn_cur() # get connection and cursor

  # Now select column names while inserting the table name into the WERE
  column_name_query =  """SELECT column_name FROM information_schema.columns
       WHERE table_name = '%s' """ %table_name

  cur.execute(column_name_query) # exectue
  my_data = cur.fetchall() # store

  cur.close() # close
  conn.close() # close

  return(my_data) # return

# Check table_names
def get_table_names():
  conn, cur = get_conn_cur() # get connection and cursor

  # query to get table names
  table_name_query = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public' """

  cur.execute(table_name_query) # execute
  my_data = cur.fetchall() # fetch results

  cur.close() #close cursor
  conn.close() # close connection

  return(my_data) # return your fetched results

## Make a SQL head function - 1 point

Make function to get the pandas equivalent of `.head()`

This function should be called `sql_head` and take a single argument of `table_name` where you specify the table name you want the head information from.  It should return the column names along with the first five rows of the table along.  

High-five if you return a pandas dataframe with this information so it displays nicely:)

In [24]:
# make sql_head function
def sql_head(table_name):
  conn, cur = get_conn_cur()
  my_query = 'SELECT * FROM ' + table_name + ' LIMIT 5;'
  ret_val = pd.read_sql_query(my_query, conn)
  return ret_val

In [25]:
# Check that it works!
sql_head(table_name = 'event')

,event_id,venue_id,cat_id,date_id,event_name,start_time
0,2,306,8,2114,Boris Godunov,2008-10-15 20:00:00
1,3,302,8,1935,Salome,2008-04-19 14:30:00
2,4,309,8,2090,La Cenerentola (Cinderella),2008-09-21 14:30:00
3,5,302,8,1982,Il Trovatore,2008-06-05 19:00:00
4,6,308,8,2109,L Elisir d Amore,2008-10-10 19:30:00


## Explore and SELECT - 1 point

Let's start this homework with some basic queries to get a look at what's in the various tables.  I want you to do the following.

* Use your get_table_names() function to see what tables are in the database.
* Use your get_column_names() to get the column names of each of the tables.  **Do this all within a single cell to keep it neat**.
* Make and run a query that selects all columns from the event table.  Only return the first 5 rows.
* Use the `sql_head()` function you created to get the first five rows of the sales table.

In [26]:
# Getting table names
get_table_names()

[('users',),
 ('category',),
 ('date',),
 ('listing',),
 ('venue',),
 ('event',),
 ('sales',)]

In [35]:
# Getting column names
print(get_column_names('users'))
print(get_column_names('category'))
print(get_column_names('date'))
print(get_column_names('listing'))
print(get_column_names('venue'))
print(get_column_names('event'))
print(get_column_names('sales'))

[('user_id',), ('user_name',), ('first_name',), ('last_name',), ('city',), ('state',), ('email',), ('phone',), ('like_sports',), ('like_theatre',), ('like_concerts',), ('like_jazz',), ('like_classical',), ('like_opera',), ('like_rock',), ('like_vegas',), ('like_broadway',), ('like_musicals',)]
[('cat_id',), ('cat_group',), ('cat_name',), ('cat_desc',)]
[('date_id',), ('cal_date',), ('day',), ('week',), ('month',), ('qtr',), ('year',), ('holiday',)]
[('list_id',), ('seller_id',), ('event_id',), ('date_id',), ('num_tickets',), ('price_per_ticket',), ('total_price',), ('list_time',)]
[('venue_id',), ('venue_name',), ('venue_city',), ('venue_state',), ('venue_seats',)]
[('event_id',), ('venue_id',), ('cat_id',), ('date_id',), ('event_name',), ('start_time',)]
[('sales_id',), ('list_id',), ('seller_id',), ('buyer_id',), ('event_id',), ('date_id',), ('qty_sold',), ('price_paid',), ('commission',), ('sale_time',)]


In [36]:
# Could also just use a list comprehension vs a bunch of print statements :)
names = get_table_names()
[get_column_names(table_name= x) for x in names]

[[('user_id',),
  ('user_name',),
  ('first_name',),
  ('last_name',),
  ('city',),
  ('state',),
  ('email',),
  ('phone',),
  ('like_sports',),
  ('like_theatre',),
  ('like_concerts',),
  ('like_jazz',),
  ('like_classical',),
  ('like_opera',),
  ('like_rock',),
  ('like_vegas',),
  ('like_broadway',),
  ('like_musicals',)],
 [('cat_id',), ('cat_group',), ('cat_name',), ('cat_desc',)],
 [('date_id',),
  ('cal_date',),
  ('day',),
  ('week',),
  ('month',),
  ('qtr',),
  ('year',),
  ('holiday',)],
 [('list_id',),
  ('seller_id',),
  ('event_id',),
  ('date_id',),
  ('num_tickets',),
  ('price_per_ticket',),
  ('total_price',),
  ('list_time',)],
 [('venue_id',),
  ('venue_name',),
  ('venue_city',),
  ('venue_state',),
  ('venue_seats',)],
 [('event_id',),
  ('venue_id',),
  ('cat_id',),
  ('date_id',),
  ('event_name',),
  ('start_time',)],
 [('sales_id',),
  ('list_id',),
  ('seller_id',),
  ('buyer_id',),
  ('event_id',),
  ('date_id',),
  ('qty_sold',),
  ('price_paid',),
  ('c

In [37]:
# Query on events
sq = "SELECT * FROM event LIMIT 5;"
run_query(sq)

(['event_id', 'venue_id', 'cat_id', 'date_id', 'event_name', 'start_time'],
 [(2, 306, 8, 2114, 'Boris Godunov', datetime.datetime(2008, 10, 15, 20, 0)),
  (3, 302, 8, 1935, 'Salome', datetime.datetime(2008, 4, 19, 14, 30)),
  (4,
   309,
   8,
   2090,
   'La Cenerentola (Cinderella)',
   datetime.datetime(2008, 9, 21, 14, 30)),
  (5, 302, 8, 1982, 'Il Trovatore', datetime.datetime(2008, 6, 5, 19, 0)),
  (6,
   308,
   8,
   2109,
   'L Elisir d Amore',
   datetime.datetime(2008, 10, 10, 19, 30))])

In [40]:
# Use sql_head to get the head of sales
sql_head('sales')

,sales_id,list_id,seller_id,buyer_id,event_id,date_id,qty_sold,price_paid,commission,sale_time
0,2,4,8117,11498,4337,1983,2,76,11.40,6/6/2008 05:00:16
1,3,5,1616,17433,8647,1983,2,350,52.50,6/6/2008 08:26:17
2,4,5,1616,19715,8647,1986,1,175,26.25,6/9/2008 08:38:52
3,5,6,47402,14115,8240,2069,2,154,23.10,8/31/2008 09:17:02
4,6,10,24858,24888,3375,2023,2,394,59.10,7/16/2008 11:59:24


## WHERE - 2 points

Now let's do a bit of filtering with WHERE.  Write and run queries to get the following results.  LIMIT all returns to first five rows. 

* Get venues with >= 10000 seats from the venues table
* Get venues in Arizona
* Get users who have a first name that starts with H
* Get just email addresses of users who gave a .edu email address




In [41]:
# Get big venues... so those with >= than 10000 seats
sq = """ SELECT * FROM venue
          WHERE venue_seats >= 10000 LIMIT 5;"""
run_query(sq)

(['venue_id', 'venue_name', 'venue_city', 'venue_state', 'venue_seats'],
 [(5, 'Gillette Stadium', 'Foxborough', 'MA', 68756),
  (6, 'New York Giants Stadium', 'East Rutherford', 'NJ', 80242),
  (15, 'McAfee Coliseum', 'Oakland', 'CA', 63026),
  (18, 'Madison Square Garden', 'New York City', 'NY', 20000),
  (67, 'Ralph Wilson Stadium', 'Orchard Park', 'NY', 73967)])

In [48]:
# Get venues in AZ
sq = """ SELECT * FROM venue
          WHERE venue_state LIKE 'AZ'LIMIT 5;"""
run_query(sq)

(['venue_id', 'venue_name', 'venue_city', 'venue_state', 'venue_seats'],
 [(38, 'US Airways Center', 'Phoenix', 'AZ', 0),
  (65, 'Jobing.com Arena', 'Glendale', 'AZ', 0),
  (92, 'University of Phoenix Stadium', 'Glendale', 'AZ', 0),
  (117, 'Chase Field', 'Phoenix', 'AZ', 0)])

In [52]:
# Get user names of who have names starting with 'H'
sq = """ SELECT user_name, first_name, last_name FROM users
          WHERE first_name LIKE 'H%'LIMIT 5;"""
run_query(sq)

(['user_name', 'first_name', 'last_name'],
 [('QTF33MCG', 'Henry', 'Cochran'),
  ('RHT62AGI', 'Hermione', 'Trevino'),
  ('HUH27PKK', 'Helen', 'Avery'),
  ('MHU11LZP', 'Howard', 'Wiley'),
  ('TWU10MZT', 'Herman', 'Myers')])

In [53]:
# Get all .edu email addresses... just the email addresses
sq = """ SELECT email FROM users
          WHERE email LIKE '%.edu'LIMIT 5;"""
run_query(sq)

(['email'],
 [('Suspendisse.tristique@nonnisiAenean.edu',),
  ('ullamcorper.nisl@Cras.edu',),
  ('vel.est@velitegestas.edu',),
  ('justo.nec.ante@quismassa.edu',),
  ('non.justo.Proin@ametconsectetuer.edu',)])

## GROUP BY and HAVING - 2 points
 
Time to practice some GROUP BY and HAVING operations.  Please write and run queries that do the following:
 
GROUP BY application
* Find the top five venues that hosted the most events.  Alias the count of events as 'events_hosted'.  Also return the venue ID
* Get the number of events hosted in each month.  You'll need to use `date_part()` in your select to select just the months.  Alias this as 'month' and then the count of the number of events hosted as 'events_hosted'. 
* Get the top five sellers who made the most commission.  Alias their total commission made as 'total_com'.  Also get their average commission made and alias as 'avg_com'.  Be sure to also display the seller_id.  
 
HAVING application
* Using the same query as the last one, instead of getting the top five sellers get all sellers who have made a total commission greater than 4000.
* Using the same query as the first groupby, instead of returning the top five venues, return just the ID's of venues that have had greater than 60 events. 

In [63]:
# Which venue_id hosted the most events?  
sq = """ SELECT venue_id, COUNT(event_id) as events_hosted FROM event
          GROUP BY venue_id
          ORDER BY events_hosted DESC
          LIMIT 5;"""
run_query(sq)

(['venue_id', 'events_hosted'],
 [(220, 81), (217, 81), (203, 80), (222, 74), (216, 72)])

In [70]:
# Get the number of events hosted each month
sq = """ SELECT date_part('month', start_time) as month, COUNT(event_id) as events_hosted FROM event
          GROUP BY month
          ORDER BY events_hosted DESC
          LIMIT 5;"""
run_query(sq)

(['month', 'events_hosted'],
 [(1.0, 777), (3.0, 753), (9.0, 746), (8.0, 737), (10.0, 735)])

In [72]:
# Get the top five sellers who made the most commission.  Also display their ID and their average commission
sq = """ SELECT seller_id, SUM(commission) as total_com, AVG(commission) as avg_com FROM sales
          GROUP BY seller_id
          ORDER BY total_com DESC
          LIMIT 5;"""
run_query(sq)

(['seller_id', 'total_com', 'avg_com'],
 [(1140, 4859.85, 347.13214285714287),
  (43551, 4704.75, 470.475),
  (13385, 4274.25, 388.5681818181818),
  (25433, 4147.949999999999, 518.4937499999999),
  (2372, 4073.8500000000004, 678.975)])

In [75]:
# Get the seller id, total commission made, and average commision made for sellers who have made more than $4000
sq = """ SELECT seller_id, SUM(commission) as total_com, AVG(commission) as avg_com FROM sales
          GROUP BY seller_id
          HAVING SUM(commission) >= 4000
          ORDER BY total_com DESC
          LIMIT 5;"""
run_query(sq)

(['seller_id', 'total_com', 'avg_com'],
 [(1140, 4859.85, 347.13214285714287),
  (43551, 4704.75, 470.475),
  (13385, 4274.25, 388.5681818181818),
  (25433, 4147.95, 518.49375),
  (2372, 4073.8500000000004, 678.975)])

In [77]:
# Get the ids of venues that have hosted over 60 events. Return just the venue_id
sq = """ SELECT venue_id FROM event
          GROUP BY venue_id
          HAVING COUNT(event_id) >= 60
          ORDER BY COUNT(event_id) DESC
          LIMIT 5;"""
run_query(sq)

(['venue_id'], [(217,), (220,), (203,), (222,), (216,)])

## JOIN - 2 points
 
Time for some joins.  You've probably noticed by now that there is at least one relational key in each table, but some have more.  For example, sales has a unique sale id, listing id, seller id, buyer id, date id.  This allows you to link each sale to relevant information in other tables.  
 
Please write queries to do the following items:
 
* Join information of users to each sale made.  
* Join information about each venue to each event. 

In [78]:
sq = """ SELECT * FROM sales
          LIMIT 5;"""
run_query(sq)

(['sales_id',
  'list_id',
  'seller_id',
  'buyer_id',
  'event_id',
  'date_id',
  'qty_sold',
  'price_paid',
  'commission',
  'sale_time'],
 [(2, 4, 8117, 11498, 4337, 1983, 2, 76, 11.4, '6/6/2008 05:00:16'),
  (3, 5, 1616, 17433, 8647, 1983, 2, 350, 52.5, '6/6/2008 08:26:17'),
  (4, 5, 1616, 19715, 8647, 1986, 1, 175, 26.25, '6/9/2008 08:38:52'),
  (5, 6, 47402, 14115, 8240, 2069, 2, 154, 23.1, '8/31/2008 09:17:02'),
  (6, 10, 24858, 24888, 3375, 2023, 2, 394, 59.1, '7/16/2008 11:59:24')])

In [79]:
sq = """ SELECT * FROM users
          LIMIT 5;"""
run_query(sq)

(['user_id',
  'user_name',
  'first_name',
  'last_name',
  'city',
  'state',
  'email',
  'phone',
  'like_sports',
  'like_theatre',
  'like_concerts',
  'like_jazz',
  'like_classical',
  'like_opera',
  'like_rock',
  'like_vegas',
  'like_broadway',
  'like_musicals'],
 [(2,
   'PGL08LJI',
   'Vladimir',
   'Humphrey',
   'Murfreesboro',
   'SK',
   'Suspendisse.tristique@nonnisiAenean.edu',
   '(783) 492-1886',
   None,
   None,
   None,
   'True',
   'True',
   None,
   None,
   'True',
   'False',
   'True'),
  (3,
   'IFT66TXU',
   'Lars',
   'Ratliff',
   'High Point',
   'ME',
   'amet.faucibus.ut@condimentumegetvolutpat.ca',
   '(624) 767-2465',
   'True',
   'False',
   None,
   'False',
   None,
   'False',
   'True',
   None,
   None,
   'True'),
  (4,
   'XDZ38RDD',
   'Barry',
   'Roy',
   'Omaha',
   'AB',
   'sed@lacusUtnec.ca',
   '(355) 452-8168',
   'False',
   'True',
   None,
   'False',
   None,
   None,
   None,
   None,
   None,
   'False'),
  (5,
   'AEB55

In [80]:
get_column_names(table_name='users')

[('user_id',),
 ('user_name',),
 ('first_name',),
 ('last_name',),
 ('city',),
 ('state',),
 ('email',),
 ('phone',),
 ('like_sports',),
 ('like_theatre',),
 ('like_concerts',),
 ('like_jazz',),
 ('like_classical',),
 ('like_opera',),
 ('like_rock',),
 ('like_vegas',),
 ('like_broadway',),
 ('like_musicals',)]

In [81]:
get_column_names(table_name='sales')

[('sales_id',),
 ('list_id',),
 ('seller_id',),
 ('buyer_id',),
 ('event_id',),
 ('date_id',),
 ('qty_sold',),
 ('price_paid',),
 ('commission',),
 ('sale_time',)]

In [87]:
# Join users information to each sale
sq = """ SELECT * FROM sales
          LEFT JOIN users ON sales.buyer_id = users.user_id
          LIMIT 5;
          """
conn, cur = get_conn_cur()
pd.read_sql_query(sq, conn)

,sales_id,list_id,seller_id,buyer_id,event_id,date_id,qty_sold,price_paid,commission,sale_time,user_id,user_name,first_name,last_name,city,state,email,phone,like_sports,like_theatre,like_concerts,like_jazz,like_classical,like_opera,like_rock,like_vegas,like_broadway,like_musicals
0,4,5,1616,19715,8647,1986,1,175,26.25,6/9/2008 08:38:52,19715,PZY80KEH,Alexander,Spencer,Jordan Valley,ID,amet@est.edu,(963) 349-0806,False,False,None,False,None,None,None,None,False,None
1,6,10,24858,24888,3375,2023,2,394,59.10,7/16/2008 11:59:24,24888,QDE27MBM,Simon,Battle,Lockport,NH,fermentum.arcu@sitametante.org,(512) 901-0259,None,None,True,True,None,True,None,None,False,False
2,8,10,24858,19715,3375,2017,1,197,29.55,7/10/2008 02:12:36,19715,PZY80KEH,Alexander,Spencer,Jordan Valley,ID,amet@est.edu,(963) 349-0806,False,False,None,False,None,None,None,None,False,None
3,9,10,24858,29891,3375,2029,3,591,88.65,7/22/2008 02:23:17,29891,YVR43ZZK,Stone,Wiggins,Sierra Vista,OK,vitae.risus.Duis@nonantebibendum.ca,(225) 417-7016,None,None,None,False,None,False,True,None,None,None
4,11,12,45635,8435,4769,2042,2,130,19.50,8/4/2008 03:06:36,8435,DAR09UFQ,Hamish,Flores,Delta Junction,NU,erat.Etiam.vestibulum@orci.org,(456) 656-0201,False,False,None,None,None,True,None,None,None,None


In [89]:
get_column_names(table_name='event')

[('event_id',),
 ('venue_id',),
 ('cat_id',),
 ('date_id',),
 ('event_name',),
 ('start_time',)]

In [90]:
get_column_names(table_name='venue')

[('venue_id',),
 ('venue_name',),
 ('venue_city',),
 ('venue_state',),
 ('venue_seats',)]

In [91]:
# For each event attach the venue information
sq = """ SELECT * FROM event
          LEFT JOIN venue ON event.venue_id = venue.venue_id
          LIMIT 5;
          """
conn, cur = get_conn_cur()
pd.read_sql_query(sq, conn)

,event_id,venue_id,cat_id,date_id,event_name,start_time,venue_id,venue_name,venue_city,venue_state,venue_seats
0,2,306,8,2114,Boris Godunov,2008-10-15 20:00:00,306,Lyric Opera House,Baltimore,MD,0
1,3,302,8,1935,Salome,2008-04-19 14:30:00,302,Detroit Opera House,Detroit,MI,0
2,4,309,8,2090,La Cenerentola (Cinderella),2008-09-21 14:30:00,309,Los Angeles Opera,Los Angeles,CA,0
3,5,302,8,1982,Il Trovatore,2008-06-05 19:00:00,302,Detroit Opera House,Detroit,MI,0
4,6,308,8,2109,L Elisir d Amore,2008-10-10 19:30:00,308,Metropolitan Opera,New York City,NY,0


## Subqueries - 2 points

To wrap up let's do several subqueries. Please do the following:

* Get all purchases made by users of live in Arizona
* Get event information for all events that took place in a venue where the venue name ends with 'Stadium'. 
* Get event information for all events where the total ticket sales were greater than $50,000.  

In [95]:
# Get all purchases from users who live in Arizona
sq = """ SELECT * FROM sales
            WHERE seller_id IN (SELECT user_id FROM users WHERE state LIKE 'AZ')
            LIMIT 5;"""
conn, cur = get_conn_cur()
pd.read_sql_query(sq, conn)

,sales_id,list_id,seller_id,buyer_id,event_id,date_id,qty_sold,price_paid,commission,sale_time
0,177,180,28510,9595,6683,2039,4,356,53.4,8/1/2008 10:04:18
1,595,604,20192,3928,2044,2122,2,364,54.6,10/23/2008 06:06:29
2,596,604,20192,705,2044,2118,3,546,81.9,10/19/2008 06:07:26
3,597,604,20192,17856,2044,2139,1,182,27.3,11/9/2008 06:12:29
4,613,621,13500,20833,5178,1990,4,504,75.6,6/13/2008 07:55:52


In [96]:
# Get event information for all events that took place in a venue where the name ended in 'Stadium'
sq = """ SELECT * FROM event
            WHERE venue_id IN (SELECT venue_id FROM venue WHERE venue_name LIKE '%Stadium')
            LIMIT 5;"""
conn, cur = get_conn_cur()
pd.read_sql_query(sq, conn)

,event_id,venue_id,cat_id,date_id,event_name,start_time
0,3803,2,9,2181,Dropkick Murphys,2008-12-21 14:00:00
1,3816,11,9,2139,Keb Mo,2008-11-09 19:00:00
2,3821,79,9,1885,Charlie Daniels Band,2008-02-28 19:30:00
3,3824,98,9,1885,Govt Mule,2008-02-28 14:00:00
4,3835,74,9,2073,LeAnn Rimes,2008-09-04 14:30:00


In [109]:
# Get event information where the total sales for that event were greater than 50000
sq = """ SELECT * FROM event
            WHERE event_id IN (SELECT event_id FROM sales WHERE (qty_sold*price_paid) > 50000)
            LIMIT 5;"""
conn, cur = get_conn_cur()
pd.read_sql_query(sq, conn)
# Wasnt exactly sure where total sales were stored, I assume it is calculated by multiplying the qty of
# tickets sold by the price they were sold at.

,event_id,venue_id,cat_id,date_id,event_name,start_time
0,2502,248,7,1920,The Country Girl,2008-04-04 15:00:00
1,7177,20,9,1904,At The Gates,2008-03-19 19:00:00


In [110]:
get_table_names()

[('users',),
 ('category',),
 ('date',),
 ('listing',),
 ('venue',),
 ('event',),
 ('sales',)]

In [111]:
get_column_names(table_name= 'date')

[('date_id',),
 ('cal_date',),
 ('day',),
 ('week',),
 ('month',),
 ('qtr',),
 ('year',),
 ('holiday',)]

In [112]:
sq = """ SELECT * FROM date
          LIMIT 5;"""
run_query(sq)

(['date_id', 'cal_date', 'day', 'week', 'month', 'qtr', 'year', 'holiday'],
 [(1828, '2008-01-02', 'TH', 1, 'JAN', 1, 2008, False),
  (1829, '2008-01-03', 'FR', 1, 'JAN', 1, 2008, False),
  (1830, '2008-01-04', 'SA', 2, 'JAN', 1, 2008, False),
  (1831, '2008-01-05', 'SU', 2, 'JAN', 1, 2008, False),
  (1832, '2008-01-06', 'MO', 2, 'JAN', 1, 2008, False)])

In [ ]:
sq = """ SELECT month, COUNT(date_id) as num_events FROM date
          GROUP BY month
          ORDER BY num_events;"""
run_query(sq)

(['month', 'num_events'],
 [('FEB', 28),
  ('SEP', 30),
  ('JUN', 30),
  ('APR', 30),
  ('JAN', 30),
  ('NOV', 30),
  ('AUG', 31),
  ('MAR', 31),
  ('OCT', 31),
  ('JUL', 31),
  ('MAY', 31),
  ('DEC', 31)])